In [1]:
from cblearn import datasets
import numpy as np
import cblearn
import tqdm
import time
import sys
sys.path.append('../..')
from comparisonHC import HandlerQuadruplets, ListQuadruplets, get_AddS_quadruplets, get_MulK_quadruplets, ComparisonHC

In [2]:
#fetching vogue cover dataset
data = datasets.fetch_vogue_cover_similarity()

In [3]:
data

{'triplet': array([[39, 10, 25],
        [55, 32, 34],
        [14, 45, 21],
        ...,
        [ 6, 12, 24],
        [28, 59, 24],
        [35, 37, 38]], dtype=int64),
 'image_label': array(['Cover_uk_VOgue_MAY10_V_29mar10_bt_268x353.jpg',
        'February_1976_covers_v_17dec10_Bt_268x353.jpg',
        'July-1978_v_2aug10_B_240x360.jpg',
        'Vogue-cover-August-1978_v_2aug10_B_240x360.jpg',
        'VogueCover1983_Jul_V_16Aug10_bt_268x353.jpg',
        'VogueCoverFeb91_XL_320x421.jpg',
        'VogueCoverJan75_V_28jul10_bt_268x353.jpg',
        'VogueCoverNov75_V_22jul10_bt_268x353.jpg',
        'VogueFeb88_V_25jan12_b_240x360.jpg',
        'VogueSep75_V_25jan12_b_240x360.jpg',
        'VoguecoverApr01gbundchen_XL_320x421.jpg',
        'VoguecoverApr04_XL_320x421.jpg', 'VoguecoverApr09_421.jpg',
        'VoguecoverApr1996_E_XL_320x421.jpg',
        'VoguecoverAug00_XL_320x421.jpg', 'VoguecoverAug03_XL_320x421.jpg',
        'VoguecoverAug06_XL_320x421.jpg', 'VoguecoverDec90_XL_3

In [4]:
#getting raw triplets from vogue cover data
triplets_raw = data['triplet']

In [5]:
n = np.amax(triplets_raw) + 1

In [6]:
#objects in the data
n

60

In [7]:
#raw triplets in the data(number of responses)
n_triplets_raw = triplets_raw.shape[0]
print(n_triplets_raw)

1107


In [8]:
#getting standard triplets from raw triplets. Random triplets are of form (i,j,k) where s_ij>s_ik
triplets = cblearn.preprocessing.triplets_from_oddoneout(data['triplet'])

In [9]:
#getting quadruplets of form (i,j,i,k) from triplets of form (i,j,k)
quadruplets = np.transpose(np.array([triplets[:,0],triplets[:,1],triplets[:,0],triplets[:,2]]))

In [10]:
n_quadruplets = quadruplets.shape[0]

In [11]:
#standard quadruplets obtained from the data
n_quadruplets

2214

In [12]:
Oracle = ListQuadruplets(quadruplets,n,n_quadruplets)

In [13]:
adds_similarities = get_AddS_quadruplets(Oracle,n)

In [14]:
#running comparison hierarchical clustering using AddS-4 similarities
chc = ComparisonHC(adds_similarities,n)
chc.fit([[j] for j in range(n)])
print("ComparisonHC ran for {:.2f} seconds.".format(chc.time_elapsed))
print("Revenue with AddS-4: {:.3e}".format(-chc.cost_dasgupta(adds_similarities)))

ComparisonHC ran for 0.73 seconds.
Revenue with AddS-4: 2.722e+04


In [15]:
mulk_similarities = get_MulK_quadruplets(Oracle,n)

In [16]:
#getting 4K-AL similarities
al4k_similarities = mulk_similarities + 2*adds_similarities

In [17]:
#running comparison hierarchical clustering using 4K-AL similarities
chc_al4k = ComparisonHC(al4k_similarities,n)
chc_al4k.fit([[j] for j in range(n)])
print("ComparisonHC ran for {:.2f} seconds.".format(chc_al4k.time_elapsed))
print("Revenue with 4K-AL: {:.3e}".format(-chc_al4k.cost_dasgupta(adds_similarities)))

ComparisonHC ran for 0.73 seconds.
Revenue with 4K-AL: 2.550e+04
